In [1]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import time
from tensorflow.keras.preprocessing import image
import random

# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

In [2]:
class Converter:
    def __init__(self, _model_name, _model_dir, _data_dir):
        # model_name = saved model directory name
        self.model_name = _model_name
        self.model_saved_name = f'{_model_name}.tflite'
        self.model_dir = _model_dir
        
        self.img_size   = 224
        self.img_height = self.img_size
        self.img_width  = self.img_size
        self.data_dir = _data_dir
        
    
    def convert(self):
        def representative_data_gen():
            dataset_list = glob.glob(f'{self.data_dir}/**/*')
            
            n = 50
            dataset_list = random.sample(dataset_list, n)
            
            for i in range(n):
                image = tf.io.read_file(dataset_list[i])
                image = tf.io.decode_jpeg(image, channels=3)
                image = tf.image.resize(image, [self.img_height, self.img_width])
                image = tf.cast(image / 255., tf.float32)
                image = tf.expand_dims(image, 0)
                yield [image]

        # path to the SavedModel directory
        self.converter = tf.lite.TFLiteConverter.from_saved_model(f'{self.model_dir}/{self.model_name}') 
        
        # This enables quantization
        self.converter.optimizations = [tf.lite.Optimize.DEFAULT]  
        
        # This sets the representative dataset for quantization
        self.converter.representative_dataset = representative_data_gen  

        # This ensures that if any ops can't be quantized, the converter throws an error
        self.converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        
        # For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
        self.converter.target_spec.supported_types = [tf.int8] 
        
        # These set the input and output tensors to uint8 (added in r2.3)
        self.converter.inference_input_type = tf.uint8
        self.converter.inference_output_type = tf.uint8
        self.tflite_model = self.converter.convert()
        
        # Save the model.
        with open(self.model_saved_name, 'wb') as f:
            f.write(self.tflite_model)


In [16]:
### custom_saved_model

In [17]:
converter = Converter('custom_saved_model', _model_dir = '../saved_models', _data_dir = '../Test_data')
converter.convert()
!edgetpu_compiler custom_saved_model.tflite

Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 266 ms.

Input model: custom_saved_model.tflite
Input size: 1.93MiB
Output model: custom_saved_model_edgetpu.tflite
Output size: 1.98MiB
On-chip memory available for caching model parameters: 4.94MiB
On-chip memory used for caching model parameters: 404.25KiB
Off-chip memory used for streaming uncached model parameters: 1.53MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 16
Operation log: custom_saved_model_edgetpu.log
See the operation log file for individual operation details.


In [3]:
### custom_resnet_saved_model

In [4]:
converter = Converter('custom_resnet_saved_model', _model_dir = '../saved_models', _data_dir = '../Test_data')
converter.convert()

!edgetpu_compiler custom_resnet_saved_model.tflite

Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 1830 ms.

Input model: custom_resnet_saved_model.tflite
Input size: 23.51MiB
Output model: custom_resnet_saved_model_edgetpu.tflite
Output size: 23.36MiB
On-chip memory available for caching model parameters: 6.25MiB
On-chip memory used for caching model parameters: 6.25MiB
Off-chip memory used for streaming uncached model parameters: 16.94MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 78
Operation log: custom_resnet_saved_model_edgetpu.log
See the operation log file for individual operation details.


In [5]:
### custom_mobilenetv2_saved_model

In [6]:
converter = Converter('custom_mobilenetv2_saved_model', _model_dir = '../saved_models', _data_dir = '../Test_data')
converter.convert()

!edgetpu_compiler custom_mobilenetv2_saved_model.tflite

Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 867 ms.

Input model: custom_mobilenetv2_saved_model.tflite
Input size: 2.60MiB
Output model: custom_mobilenetv2_saved_model_edgetpu.tflite
Output size: 2.77MiB
On-chip memory available for caching model parameters: 6.91MiB
On-chip memory used for caching model parameters: 2.71MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 71
Operation log: custom_mobilenetv2_saved_model_edgetpu.log
See the operation log file for individual operation details.


In [ ]:
## error

In [ ]:
### custom_efficientnetb2_saved_model

In [3]:
converter = Converter('custom_efficientnetb2_saved_model', _model_dir = '../saved_models', _data_dir = '../Test_data')
converter.convert()

!edgetpu_compiler custom_efficientnetb2_saved_model.tflite

Edge TPU Compiler version 2.0.291256449
Invalid model: custom_efficientnetb2_saved_model.tflite
Model not quantized


In [4]:
### custom_efficientnetb1_saved_model

In [5]:
converter = Converter('custom_efficientnetb1_saved_model', _model_dir = '../saved_models', _data_dir = '../Test_data')
converter.convert()

!edgetpu_compiler custom_saved_model.tflite

Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 233 ms.

Input model: custom_saved_model.tflite
Input size: 1.93MiB
Output model: custom_saved_model_edgetpu.tflite
Output size: 1.98MiB
On-chip memory available for caching model parameters: 4.94MiB
On-chip memory used for caching model parameters: 404.25KiB
Off-chip memory used for streaming uncached model parameters: 1.53MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 16
Operation log: custom_saved_model_edgetpu.log
See the operation log file for individual operation details.
